### Summary

The idea here is to design a gradient boosting prediction algorithm using tensorflow. We will be using estimator API, which allows us seamless transition to other models !

In [158]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import shutil

In [159]:
# The directory for storing metadata related to the model. This way, we can easily build models step by step, starting
# from the previous checkpoint, in cases when we have large amount of data.
OUTDIR = 'sample_model_metadata'

In [160]:
#  The pandas input function which is to be fed to the estimator API. NOTE : Since the syntax is very similar, we could 
# have used one function for training and evaluation as well, but we have decided to keep them separately for the sake
# of clarity.
def make_train_input_fn(df, num_epochs=1, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df['item_cnt_month'],
        batch_size=128,
        num_epochs=num_epochs,
        shuffle=shuffle,
        queue_capacity=2000)

def make_eval_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df['item_cnt_month'],
        batch_size=128,
        num_epochs=1,
        shuffle=False,
        queue_capacity=2000)

def make_prediction_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        batch_size=128,
        num_epochs=1,
        shuffle=False,
        queue_capacity=2000)

In [161]:
REL_PREDICTOR_COLS = ['date_block_num', 'shop_id', 'item_id']

In [162]:
# This routine makes the input in a tensorflow digestible form.
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in REL_PREDICTOR_COLS]
  return input_columns


##### Get relevant data into pandas dataframes.

In [163]:
sales_train = pd.read_csv("input/sales_train.csv")

In [164]:
monthly_sales_data = sales_train[['date_block_num', 'shop_id', 'item_id', 'item_cnt_day' ]].groupby(
    ['date_block_num', 'shop_id', 'item_id']).sum()
monthly_sales_data.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)
monthly_sales_data.reset_index(['date_block_num', 'shop_id', 'item_id'], inplace=True)

In [165]:
monthly_sales_data.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month'], dtype='object')

In [166]:
train_data = monthly_sales_data[monthly_sales_data.date_block_num != 32]
validation_data = monthly_sales_data[monthly_sales_data.date_block_num == 32]

##### Addtitional specific tensorflow functions.

In [167]:
# Train any model, for which a tensorflow estimator API is built. This way, we can test out several models with a one
# line change.
def get_trained_model(model, train_data, outdir=OUTDIR, logging_mode=tf.logging.INFO):
    
    tf.logging.set_verbosity(logging_mode)
    
    # Delete the directory corresponding to metadata of model so as to build it from scratch.
    shutil.rmtree(outdir, ignore_errors=True)

    # Train data for a reasonable number of epochs (100) as default.
    model.train(make_train_input_fn(train_data, num_epochs=100, shuffle=False))
    
    return model

In [168]:
def get_predictions_from_model(model, test_data):
    predictions = model.predict(input_fn=make_prediction_input_fn(test_data)) 
    return [x['predictions'][0] for x in predictions]

##### Train the model and obtain validation score. Also, make score that the predictions are clipped so that they make sense

In [169]:
model = get_trained_model(tf.estimator.BoostedTreesRegressor(feature_columns=make_feature_cols(),
                                                             n_batches_per_layer=2,
                                                             model_dir=OUTDIR),
                         train_data)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'sample_model_metadata', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13f5c6f98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph 

In [170]:
model_predictions = get_predictions_from_model(model, validation_data)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from sample_model_metadata/model.ckpt-1200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [171]:
def get_rmse(model_predictions, validation_data):
    actual_vals = np.clip(validation_data.item_cnt_month.values, 0, 20)
    return np.sqrt(mean_squared_error(actual_vals, np.clip(model_predictions, 0, 20)))

In [172]:
get_rmse(model_predictions, validation_data)

2.6254074716990776

##### What next ?

One option for us would be to add relevant data from other files and see if we can extract something more from this model.

In [173]:
shops = pd.read_csv('input/shops.csv')

In [174]:
shops.columns

Index(['shop_name', 'shop_id'], dtype='object')

In [175]:
item_categories = pd.read_csv('input/item_categories.csv')

In [176]:
item_categories.columns

Index(['item_category_name', 'item_category_id'], dtype='object')

In [177]:
items = pd.read_csv('input/items.csv')

In [178]:
items.columns

Index(['item_name', 'item_id', 'item_category_id'], dtype='object')

##### Use item_category_id as well.